# SVHN

## Setup
### Preparation for Google Colab

In [1]:
google_colab = False

In [2]:
# install
try:
    import binarybrain as bb
except:
    if google_colab:
        !pip install pybind11
        %cd /content
        !nvcc -V
        !sudo rm -fr BinaryBrain
        !rm -fr BinaryBrain
        !git clone --recursive -b ver3_develop https://github.com/ryuz/BinaryBrain.git
        %cd /content/BinaryBrain/python
        !sh copy_src.sh
        !python3 setup.py build
        !python3 setup.py develop

        import binarybrain as bb

In [3]:
# mount
if google_colab:
    from google.colab import drive 
    drive.mount('/content/drive')
    %cd /content/drive/My Drive

### setup GPU

In [4]:
# select CUDA device
device = 1
bb.set_device(device)
prop = bb.get_device_properties(device)
print(prop)

# setup OpenMP
bb.omp_set_num_threads(4)


name                     : GeForce GT 1030
totalGlobalMem           : 2147483648
sharedMemPerBlock        : 49152
regsPerBlock             : 65536
warpSize                 : 32
memPitch                 : 2147483647
maxThreadsPerBlock       : 1024
maxThreadsDim[0]         : 1024
maxThreadsDim[1]         : 1024
maxThreadsDim[2]         : 64
maxGridSize[0]           : 2147483647
maxGridSize[1]           : 65535
maxGridSize[2]           : 65535
clockRate                : 1468000
totalConstMem            : 65536
major                    : 6
minor                    : 1
textureAlignment         : 512
deviceOverlap            : 1
multiProcessorCount      : 3
kernelExecTimeoutEnabled : 1
integrated               : 0
canMapHostMemory         : 1
computeMode              : 0




## Main code

In [5]:
import binarybrain as bb
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
import os
import sys
import scipy.io as sio
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import OrderedDict

In [6]:
network_name = 'SvhnCnnLutNet'
mini_batch_size = 32

In [7]:
def plot_image(img):
    img = img.reshape(3, 32, 32).transpose(1, 2, 0)
    plt.imshow(img)

## Train
### setup tain data

In [8]:
train_data = sio.loadmat('train_32x32.mat')
test_data  = sio.loadmat('test_32x32.mat')

train_len = len(train_data['y'])
test_len = len(test_data['y'])

td = {}
td['x_train'] = train_data['X'].transpose(3, 2, 0, 1).reshape(-1, 3*32*32).astype(np.float32) / 255.0
td['x_test']  = test_data['X'].transpose(3, 2, 0, 1).reshape(-1, 3*32*32).astype(np.float32) / 255.0
td['t_train'] = np.zeros((train_len, 10), dtype=np.float32)
td['t_test']  = np.zeros((test_len, 10), dtype=np.float32)
for i in range(train_len):
    td['t_train'][i][train_data['y'][i][0] % 10] = 1.0
for i in range(test_len):
    td['t_test'][i][test_data['y'][i][0] % 10] = 1.0
td['x_shape'] = [32, 32, 3]
td['t_shape'] = [10]

In [9]:
td = bb.load_cifar10()

## Buld network

### define classes for distillation

In [10]:
def create_conv_layer(shape, w, h, batch_norm=False, act=True, padding='valid'):
    sub_net = bb.Sequential.create()
    sub_net.add(bb.DenseAffine.create(shape))
    if batch_norm:
        sub_net.add(bb.BatchNormalization.create())
    if act:
        sub_net.add(bb.ReLU.create())
    return bb.LoweringConvolution.create(sub_net, w, h, 1, 1, padding=padding)

In [11]:
net = bb.Sequential.create()

net.add(create_conv_layer([36], 3, 3))  # 32x32->30x30
net.add(create_conv_layer([72], 3, 3))  # 30x30->28x28
net.add(bb.MaxPooling.create(2, 2))     # 28x28->14x14
net.add(create_conv_layer([72], 3, 3))  # 14x14->12x12
net.add(create_conv_layer([144], 3, 3)) # 12x12->10x10
net.add(bb.MaxPooling.create(2, 2))     # 10x10->5x5
net.add(create_conv_layer([144], 3, 3)) # 5x5->3x3
net.add(create_conv_layer([288], 3, 3)) # 3x3->1x1
net.add(create_conv_layer([576], 1, 1))
net.add(create_conv_layer([10], 1, 1, act=False))

net.set_input_shape(td['x_shape'])

[1, 1, 10]

In [12]:
print(net.get_info())

----------------------------------------------------------------------
[Sequential] 
  --------------------------------------------------------------------
  [LoweringConvolution] 
   filter size : (3, 3)
   input  shape : {32, 32, 3}   output shape : {30, 30, 36}
    ------------------------------------------------------------------
    [ConvolutionIm2Col] 
     input  shape : {32, 32, 3}     output shape : {3, 3, 3}
    ------------------------------------------------------------------
    [Sequential] 
      ----------------------------------------------------------------
      [DenseAffine] 
       input  shape : {3, 3, 3}       output shape : {36}
      ----------------------------------------------------------------
      [ReLU] 
       input  shape : {36}       output shape : {36}
    ------------------------------------------------------------------
    [ConvolutionCol2Im] 
     input  shape : {36}     output shape : {30, 30, 36}
  ----------------------------------------------

In [ ]:
loss    = bb.LossSoftmaxCrossEntropy.create()
metrics = bb.MetricsCategoricalAccuracy.create()
optimizer = bb.OptimizerAdam.create()

optimizer.set_variables(net.get_parameters(), net.get_gradients())
runner = bb.Runner(net, network_name, loss, metrics, optimizer)

runner.fitting(td, epoch_size=10, mini_batch_size=mini_batch_size, file_write=True, file_read=False)

epoch=1 test_accuracy=0.515000 test_loss=1.325222 train_accuracy=0.529340 train_loss=1.274543


epoch=2 test_accuracy=0.612800 test_loss=1.098144 train_accuracy=0.639340 train_loss=1.004876


 61%|██████████████████████████████▌                   | 1913/3125 [00:45<00:27, 44.00it/s, loss=0.862, accuracy=0.695]